In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [52]:
return_frame = pd.DataFrame([])

In [53]:

for filename in os.listdir("../data/RawData/BLS"):
    return_frame = pd.concat([return_frame,pd.read_csv("../data/RawData/BLS/"+filename).groupby("Year").agg(np.mean).rename(columns={"Value":filename.split("_")[0]})],axis=1)

In [2]:
def toQuarter(month):
    if month>=1 and month<=3:
        return 1
    elif month > 3 and month <=6:
        return 2
    elif month > 6 and month <= 9:
        return 3
    else: 
        return 4

In [15]:
def formatData(filename):
    data = pd.read_csv('../data/RawData/BLS/'+filename,usecols=["Year","Period","Value"])
    
    data.Period = data.Period.apply(lambda x: x[1:])
    data.Period = data.Period.astype("int")
    data["Quarter"] = data.Period.apply(toQuarter)
    data = data.drop(columns=["Period"])
    data = data.groupby(["Year","Quarter"]).mean()
    return data

In [39]:
training = pd.read_csv("../data/TrainingData/fulldata.csv")

In [40]:

for filename in os.listdir("../data/RawData/BLS"):
    to_merge = formatData(filename).reset_index(level=[0,1])
    to_merge.rename(columns={"Value":filename.split("_")[0]},inplace=True)
    training = training.merge(to_merge,how="left",left_on=["YEAR FORECAST MADE","QUARTER"],right_on=["Year","Quarter"],suffixes=('', '_drop')).fillna(method="ffill")
training.drop([col for col in training.columns if 'drop' in col], axis=1, inplace=True)
training.drop(columns = ["Year","Quarter"],inplace=True)

In [41]:
training

,YEAR FORECAST MADE,QUARTER,YEAR BEING FORECAST,INDICATOR,FORECASTER ID,INDUSTRY,Year.ID.ForecastYear.Quarter,Indicator.Year.ID.ForecastYear.Quarter,BIN 1,BIN 2,...,ACTUAL_CONF,HIT,pred_average,pred_var,chicken,coffee,eggs,electricity,rice,unleadedGasoline
0,1968,4,1968,NominalGNP,1,NaN,1968-1-1968-4,NominalGNP-1968-1-1968-4,0.0,0.0,...,0.00,0.0,6.000,0.450000,NaN,NaN,NaN,NaN,NaN,NaN
1,1968,4,1968,NominalGNP,2,NaN,1968-2-1968-4,NominalGNP-1968-2-1968-4,0.0,0.0,...,0.00,0.0,6.100,1.040000,NaN,NaN,NaN,NaN,NaN,NaN
2,1968,4,1968,NominalGNP,3,NaN,1968-3-1968-4,NominalGNP-1968-3-1968-4,0.0,0.0,...,0.00,0.0,6.800,0.410000,NaN,NaN,NaN,NaN,NaN,NaN
3,1968,4,1968,NominalGNP,4,NaN,1968-4-1968-4,NominalGNP-1968-4-1968-4,0.0,0.0,...,0.00,0.0,6.150,1.327500,NaN,NaN,NaN,NaN,NaN,NaN
4,1968,4,1968,NominalGNP,5,NaN,1968-5-1968-4,NominalGNP-1968-5-1968-4,3.0,4.0,...,0.04,0.0,4.650,7.227500,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36057,2021,2,2024,Unemployment,586,2.0,2024-586-2021-2,Unemployment-2024-586-2021-2,0.0,0.0,...,0.00,0.0,5.514,0.578179,1.5005,4.659,1.6225,0.1395,0.793,2.9055
36058,2021,2,2024,Unemployment,587,2.0,2024-587-2021-2,Unemployment-2024-587-2021-2,0.0,0.0,...,0.00,0.0,5.125,0.203125,1.5005,4.659,1.6225,0.1395,0.793,2.9055
36059,2021,2,2024,Unemployment,588,1.0,2024-588-2021-2,Unemployment-2024-588-2021-2,0.0,0.0,...,0.00,0.0,5.125,0.203125,1.5005,4.659,1.6225,0.1395,0.793,2.9055
36060,2021,2,2024,Unemployment,589,2.0,2024-589-2021-2,Unemployment-2024-589-2021-2,0.0,0.0,...,0.00,0.0,5.175,0.050625,1.5005,4.659,1.6225,0.1395,0.793,2.9055


In [42]:
training.to_csv("../data/TrainingData/trainingWithItems.csv")